In [ ]:
import numpy as np
import os
from components.network import Network
from components.helper import Helper

positives = Helper.get_positives(range_of_samples=range(1, 500))
filepath = os.path.join(os.path.pardir, 'data/train_test_file_small.npz')
np.savez(filepath, positives=positives)

In [3]:
import os
import numpy as np
import librosa
from components.helper import Helper

filepath = os.path.join(os.path.pardir, 'data/train_test_file_small.npz')
file = np.load(filepath, allow_pickle=True)
positives = file['positives']
negatives = Helper.get_negatives(500, sample_length=660719)
train, test = Helper.prepare_dataset(positives, negatives, train_to_test_ratio=0.8)

features = [librosa.feature.melspectrogram, librosa.feature.tempogram, librosa.feature.rms, librosa.feature.mfcc,
            librosa.feature.chroma_stft]

train_features = Helper.get_data_list_of_all_features(features, train)
test_features = Helper.get_data_list_of_all_features(features, test)

Number of all negative samples: 6183
Length of the specimen should be 660719
Started feature: <function melspectrogram at 0x0000020715E6DD30>
Finished feature: <function melspectrogram at 0x0000020715E6DD30> in 32.180219650268555s
Started feature: <function tempogram at 0x0000020715E6DE50>
Finished feature: <function tempogram at 0x0000020715E6DE50> in 63.27040982246399s
Started feature: <function rms at 0x0000020715E6D790>
Finished feature: <function rms at 0x0000020715E6D790> in 19.051026105880737s
Started feature: <function mfcc at 0x0000020715E6DCA0>
Finished feature: <function mfcc at 0x0000020715E6DCA0> in 24.305002689361572s
Started feature: <function chroma_stft at 0x0000020715E6D9D0>
Finished feature: <function chroma_stft at 0x0000020715E6D9D0> in 35.44700002670288s
Started feature: <function melspectrogram at 0x0000020715E6DD30>
Finished feature: <function melspectrogram at 0x0000020715E6DD30> in 7.610013246536255s
Started feature: <function tempogram at 0x0000020715E6DE50>


In [ ]:
from components.network import Network

networks = []
for feature in features:
    networks.append(Network(input_shape=train_features[feature][0][0].shape))

In [1]:
import os
from components.network import Network
import librosa

features = [librosa.feature.melspectrogram, librosa.feature.tempogram, librosa.feature.rms, librosa.feature.mfcc,
            librosa.feature.chroma_stft]

networks = [Network(input_shape=(128, 1291)),
            Network(input_shape=(384, 1291)),
            Network(input_shape=(1, 1291)),
            Network(input_shape=(20, 1291)),
            Network(input_shape=(12, 1291))]

for i, network in enumerate(networks):
    feature_name = str(features[i]).split(' ')[1]
    network.load_model(os.path.join(os.path.pardir, f'models/model_{feature_name}'))

Using plaidml.keras.backend backend.
INFO:plaidml:Opening device "opencl_amd_gfx1010.0"


In [4]:
for i, feature in enumerate(features):
    train_set = train_features[feature]
    x_train, y_train = list(zip(*train_set))
    networks[i].train(x_train=np.array(x_train), y_train=np.array(y_train), epochs=50, batch_size=100)


Epoch 1/50
800/800 [==============================] - 34s 43ms/step - loss: 31.7781 - acc: 0.9212
Epoch 2/50
800/800 [==============================] - 0s 539us/step - loss: 0.4424 - acc: 0.9925

In [5]:
import os

for i, network in enumerate(networks):
    feature_name = str(features[i]).split(' ')[1]
    network.save_model(os.path.join(os.path.pardir, f'models/model_{feature_name}'))

In [ ]:
import librosa

from components.specimen import Evolution, Specimen
from components.helper import Helper

#specimen_length = 660984
specimen_length = 660719
evolution = Evolution(neural_networks=networks,
                      features_list=[librosa.feature.melspectrogram, librosa.feature.tempogram, librosa.feature.rms,
                                     librosa.feature.mfcc, librosa.feature.chroma_stft],
                      number_of_specimen=100,
                      specimen_length=specimen_length, mutation_chance=0.1, crossover_chance=0.1)
supplement = Helper.get_positives(range_of_samples=range(505, 507), sample_length=660719)
for i, specimen in enumerate(supplement):
    evolution.inject_specimen(Specimen(features=specimen[0]), i)
evolution.run_epochs(number_of_epochs=100, save=True, save_as_negative=False, epsilon=0.7)

Generated 100 specimen of 100
Started fitness calculation
Started specimen conversion
Finished specimen conversion
Finished network 1/5 in 0.149993896484375s
Finished network 2/5 in 0.5319998264312744s
Finished network 3/5 in 0.008991718292236328s
Finished network 4/5 in 0.029999732971191406s
Finished network 5/5 in 0.020985126495361328s
Finished fitness calculation
Length of the specimen should be 660719
Started epoch 1
Creating new specimens........................................................................
Finished all specimens
Started fitness calculation
Started specimen conversion
Finished specimen conversion
Finished network 1/5 in 0.14301538467407227s
Finished network 2/5 in 0.5239849090576172s
Finished network 3/5 in 0.009001016616821289s
Finished network 4/5 in 0.031998395919799805s
Finished network 5/5 in 0.019999980926513672s
Finished fitness calculation
Best fitness: 0.9607979655265808
Worst fitness: 0.16478842496871948
Started epoch 2
Creating new specimens..........

In [ ]:
evolution.run_epochs(number_of_epochs=100, save=True, save_as_negative=False, epsilon=0.1)

